In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

In [2]:
# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


## Load and prepare data

In [3]:
# Load the data
data = pd.read_csv("../dataset.csv")

In [4]:
# Separate features and target
X = data.drop("Label", axis=1)
y = data["Label"]

In [5]:
# Convert categorical data to numeric if necessary
X = pd.get_dummies(X, sparse=True)

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1059: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  if np.may_share_memory(array, array_orig):
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy arr

## Build an AutoEncoder

In [9]:
# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [10]:
class VAELossLayer(Layer):
    """Custom layer to add VAE loss to the model."""

    def __init__(self, **kwargs):
        super(VAELossLayer, self).__init__(**kwargs)

    def call(self, inputs):
        x, x_decoded_mean, z_mean, z_log_var = inputs
        xent_loss = binary_crossentropy(x, x_decoded_mean) * X_train_scaled.shape[1]
        kl_loss = -0.5 * K.sum(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1
        )
        vae_loss = K.mean(xent_loss + kl_loss)
        self.add_loss(vae_loss)
        return x_decoded_mean

In [11]:
# Modify the autoencoder architecture
input_layer = Input(shape=(X_train_scaled.shape[1],))
x = Dense(128, activation="relu")(input_layer)
z_mean = Dense(64, activation="linear")(x)
z_log_var = Dense(64, activation="linear")(x)
z = Lambda(sampling, output_shape=(64,))([z_mean, z_log_var])

In [12]:
# Decoder
decoder_h = Dense(128, activation="relu")
decoder_mean = Dense(X_train_scaled.shape[1], activation="sigmoid")
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
autoencoder = Model(input_layer, x_decoded_mean)

In [14]:
# Use custom VAELossLayer
output_with_loss = VAELossLayer()([input_layer, x_decoded_mean, z_mean, z_log_var])

In [15]:
autoencoder = Model(input_layer, output_with_loss)
autoencoder.compile(optimizer="adam")

In [17]:
# Prediction model
predictor = Dense(1, activation="sigmoid")(z)
prediction_model = Model(input_layer, predictor)
prediction_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [19]:
# Model summaries
autoencoder.summary()
prediction_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20090)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  2,571,648 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 64)        │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │      8,320 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 20090)     │  2,591,610 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vae_loss_layer      │ (None, 20090)     │          0 │ input_layer[0][0… │
│ (VAELossLayer)      │                   │            │ dense_4[0][0],    │
│                     │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,188,090 (19.79 MB)

 Trainable params: 5,188,090 (19.79 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20090)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  2,571,648 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 64)        │          0 │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         65 │ lambda[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,588,225 (9.87 MB)

 Trainable params: 2,588,225 (9.87 MB)

 Non-trainable params: 0 (0.00 B)

## Train and evaluate

In [20]:
# Training
autoencoder.fit(X_train_scaled, epochs=30, batch_size=256, validation_split=0.2)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - loss: 9412.2998 - val_loss: 749.1769
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - loss: 532645.7500 - val_loss: 395.0850
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 870.8795 - val_loss: 686.0958
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - loss: -22.5725 - val_loss: 1072.6433
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: -810.4473 - val_loss: 2969.1572
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: -4288.9780 - val_loss: 15412.6260
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: -27319.1738 - val_loss: 78704.9531
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: -138338.0938 - val_loss: 232157.4062
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - loss: -522216.4062 - val_loss: 387974.1250
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: -1765326.2500 - val_loss: 433043.9062
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - loss: -4405529.00

In [21]:
prediction_model.fit(
    X_train_scaled, y_train, epochs=30, batch_size=256, validation_split=0.2
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.4877 - loss: 355.3042 - val_accuracy: 0.4575 - val_loss: 0.9706
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5095 - loss: 24.1428 - val_accuracy: 0.4944 - val_loss: 0.9036
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5566 - loss: 6.4610 - val_accuracy: 0.5369 - val_loss: 0.8378
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6873 - loss: 2.4616 - val_accuracy: 0.5031 - val_loss: 0.8755
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7844 - loss: 1.2079 - val_accuracy: 0.5063 - val_loss: 0.8568
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8449 - loss: 0.4570 - val_accuracy: 0.5031 - val_loss: 0.8647
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8880 - loss: 0.2820 - val_accuracy: 0.5088 - val_loss: 0.8592
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8970 - loss: 0.2218 - val_accuracy: 0.4819 

In [22]:
# Evaluate
test_loss, test_acc = prediction_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5923 - loss: 0.5855
Test Accuracy: 58.30%
